In [1]:
import gdown
file_id = '10ReMA0bgGSMsNWyCWZ1i4kHaT2j-_Lle'
url='https://drive.google.com/uc?id={}'.format(file_id)
output= 'dogbreeds.zip'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=10ReMA0bgGSMsNWyCWZ1i4kHaT2j-_Lle
To: /content/dogbreeds.zip
100%|██████████| 724M/724M [00:07<00:00, 92.8MB/s]


'dogbreeds.zip'

In [2]:
import zipfile
local_zip = 'dogbreeds.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/dogbreeds')
zip_ref.close()

In [3]:
pip install --upgrade tensorflow

In [5]:
import os
import numpy as np
import pandas as pd
import cv2
from glob import glob

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split

def build_model(size, num_classes):
    inputs = Input((size, size, 3))
    backbone = InceptionV3(input_tensor=inputs, include_top=False, weights="imagenet")
    backbone.trainable = True
    x = backbone.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.2)(x)
    x = Dense(1024, activation="relu")(x)
    x = Dense(num_classes, activation="softmax")(x)

    model = tf.keras.Model(inputs, x)
    return model

def read_image(path, size):
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (size, size))
    image = image / 255.0
    image = image.astype(np.float32)
    return image

def parse_data(x, y):
    x = x.decode()

    num_class = 120
    size = 224

    image = read_image(x, size)
    label = [0] * num_class
    label[y] = 1
    label = np.array(label)
    label = label.astype(np.int32)

    return image, label

def tf_parse(x, y):
    x, y = tf.numpy_function(parse_data, [x, y], [tf.float32, tf.int32])
    x.set_shape((224, 224, 3))
    y.set_shape((120))
    return x, y

def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    return dataset

if True:
    path = "/content/dogbreeds/"
    train_path = os.path.join(path, "train/*")
    test_path = os.path.join(path, "test/*")
    labels_path = os.path.join(path, "labels.csv")

    labels_df = pd.read_csv(labels_path)
    breed = labels_df["breed"].unique()
    print("Number of Breed: ", len(breed))

    breed2id = {name: i for i, name in enumerate(breed)}

    ids = glob(train_path)
    labels = []

    for image_id in ids:
        image_id = image_id.split("/")[-1].split(".")[0]
        breed_name = list(labels_df[labels_df.id == image_id]["breed"])[0]
        breed_idx = breed2id[breed_name]
        labels.append(breed_idx)

    ids = ids[:1000]
    labels = labels[:1000]

    ## Spliting the dataset
    train_x, valid_x = train_test_split(ids, test_size=0.2, random_state=42)
    train_y, valid_y = train_test_split(labels, test_size=0.2, random_state=42)

    ## Parameters
    size = 224
    num_classes = 120
    lr = 1e-4
    batch = 16
    epochs = 4

    ## Model
    model = build_model(size, num_classes)
    model.compile(loss="categorical_crossentropy", optimizer=Adam(lr), metrics=["acc"])
    # model.summary()

    ## Dataset
    train_dataset = tf_dataset(train_x, train_y, batch=batch)
    valid_dataset = tf_dataset(valid_x, valid_y, batch=batch)

    ## Training
    callbacks = [
        ModelCheckpoint("model.h5", verbose=1, save_best_only=True),
        ReduceLROnPlateau(factor=0.1, patience=5, min_lr=1e-6)
    ]
    train_steps = (len(train_x)//batch) + 1
    valid_steps = (len(valid_x)//batch) + 1
    model.fit(train_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        validation_data=valid_dataset,
        epochs=epochs,
        callbacks=callbacks)

Number of Breed:  120
Epoch 1/4
51/51 [==============================] - ETA: 0s - loss: 4.6466 - acc: 0.0650
Epoch 00001: val_loss improved from inf to 4.14870, saving model to model.h5
51/51 [==============================] - 30s 424ms/step - loss: 4.6466 - acc: 0.0650 - val_loss: 4.1487 - val_acc: 0.1650 - lr: 1.0000e-04
Epoch 2/4
51/51 [==============================] - ETA: 0s - loss: 2.7266 - acc: 0.6091
Epoch 00002: val_loss improved from 4.14870 to 3.30173, saving model to model.h5
51/51 [==============================] - 20s 390ms/step - loss: 2.7266 - acc: 0.6091 - val_loss: 3.3017 - val_acc: 0.2900 - lr: 1.0000e-04
Epoch 3/4
51/51 [==============================] - ETA: 0s - loss: 1.0740 - acc: 0.9338
Epoch 00003: val_loss improved from 3.30173 to 3.00267, saving model to model.h5
51/51 [==============================] - 20s 391ms/step - loss: 1.0740 - acc: 0.9338 - val_loss: 3.0027 - val_acc: 0.3550 - lr: 1.0000e-04
Epoch 4/4
51/51 [==============================] - ETA: 0s